In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from scipy.stats import norm
from sklearn import preprocessing, metrics
import keras
from keras import layers
from keras.models import Model
from keras import metrics
from keras import backend as K   # 'generic' backend so code works with either tensorflow or theano
from keras.layers import Input, Dense, Lambda, BatchNormalization, Layer
from sklearn.utils import shuffle

Using TensorFlow backend.


In [1]:
K.clear_session()

np.random.seed(237)

#, index_col=0
csv = pd.read_csv(r"\data\new_dataset3_train3.csv", index_col=0)
print (csv.shape)

data = csv.astype('float32').copy()
#data = shuffle(data)


csv2 = pd.read_csv(r"\data\new_dataset3_test3.csv", index_col=0)
print (csv2.shape)
csv2 = csv2.fillna(0)


data2 = csv2.astype('float32').copy()
#data2 = shuffle(data2)

new_data = data.append(data2)

column = new_data.columns.values.tolist()



print (new_data.isnull().values.any())

new_data.shape

data_norm = (data.ix[:, 0:-1] - new_data.ix[:, 0:-1].mean()) / (new_data.ix[:, 0:-1].max() - new_data.ix[:, 0:-1].min())
data_df =  pd.concat([data_norm, data.ix[:, -1]], axis=1)

#data_df = shuffle(data_df)

print (data_df.head(5))

data_norm2 = (data2.ix[:, 0:-1] - new_data.ix[:, 0:-1].mean()) / (new_data.ix[:, 0:-1].max() - new_data.ix[:, 0:-1].min())
data_df2 =  pd.concat([data_norm2, data2.ix[:, -1]], axis=1)

#data_df = shuffle(data_df)

print (data_df2.head(5))


data_df.to_csv('new_dataset3_train3_normalized.csv', encoding='utf-8', sep=',', index= False)
data_df2.to_csv('new_dataset3_test3_normalized.csv', encoding='utf-8', sep=',', index= False)


data_norm3 = (new_data.ix[:, 0:-1] - new_data.ix[:, 0:-1].mean()) / (new_data.ix[:, 0:-1].max() - new_data.ix[:, 0:-1].min())
data_df3 =  pd.concat([data_norm3, new_data.ix[:, -1]], axis=1)


data_df3 = shuffle(data_df3)
data_df3.head(5)


data_df3.shape

In [2]:
#data.head(n=5)
#normal_df = data_df[data_df.fraudRisk == 0] #save normal_df observations into a separate df
fraud_df = data_df3[data_df3.TARGET == 1] #do the same for frauds

fraud_df=fraud_df.fillna(0)

print (fraud_df.head(10))
print (fraud_df.shape)


print (fraud_df.isnull().values.any())

test_split = 0.2 #portion of data used for testing
val_split = 0.1 #portion of training data used for validation

first_test = int(fraud_df.shape[0] * (1 - test_split))
print (first_test)
first_val = int(first_test * (1 - val_split))
print (first_val)

train = fraud_df.iloc[:first_val]
print (train.shape)
val = fraud_df.iloc[first_val:first_test]
print (val.shape)
test = fraud_df
print (test.shape)

x_train_df, x_val_df, x_test_df = train.iloc[:, :-1], val.iloc[:, :-1], test.iloc[:, :-1]
y_train_df, y_val_df, y_test_df = train.iloc[:, -1], val.iloc[:, -1], test.iloc[:, -1]
print(x_train_df.shape)
print(x_train_df.head(5))


x_train, x_val, x_test = x_train_df.values, x_val_df.values, x_test_df.values
y_train, y_val, y_test = y_train_df.values, y_val_df.values, y_test_df.values

scaler = preprocessing.StandardScaler()

x_train, x_val, x_test = scaler.fit_transform(x_train), scaler.fit_transform(x_val), scaler.fit_transform(x_test)


fraud_df.isnull().values.any() 


In [9]:
from keras.optimizers import Adam 
from keras.optimizers import Adadelta
from keras.optimizers import RMSprop
from keras.optimizers import Adamax
from keras.layers import LSTM


In [3]:
class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs
    


hidden_size = 123 #size of the hidden layer in encoder and decoder
hidden_size2 = 60 
hidden_size3 = 30
hidden_size4 = 15
hidden_size5 = 7
latent_dim = 2 #number of latent variables to learn
input_dim = x_train.shape[1]

x = Input(shape=(input_dim,))

t = BatchNormalization()(x)
t = Dense(hidden_size, activation='tanh' , name='encoder_hidden5')(t)
#t = BatchNormalization()(t)
t = Dense(hidden_size2, activation='tanh' , name='encoder_hidden4')(t)
t = Dense(hidden_size3, activation='tanh' , name='encoder_hidden3')(t)
t = Dense(hidden_size4, activation='tanh' , name='encoder_hidden2')(t)
t = Dense(hidden_size5, activation='tanh' , name='encoder_hidden')(t)



z_mean = Dense(latent_dim, name='z_mean')(t)
z_log_var = Dense(latent_dim, name='z_log_var')(t)
z_mean, z_log_var = KLDivergenceLayer()([z_mean, z_log_var])

#, mean=0., stddev=1.
#numpy ramdom integer

In [11]:
def sampling(args):
    z_mean, z_log_var = args
    #batch = K.shape(z_mean)[0] #new
    #dim = K.int_shape(z_mean)[1] #new
    #epsilon = K.random_normal(shape=(batch, dim)) #new
    epsilon = K.random_normal(shape=K.shape(z_mean), mean=0. , stddev= 1. )
    return z_mean + K.exp(z_log_var) * epsilon





z = Lambda(sampling, name='z_sampled')([z_mean, z_log_var])
#t = BatchNormalization()(z)
encoder = Model(x, [z_mean, z_log_var, z])
encoder.summary()

latent_inputs = Input(shape=(latent_dim,), name='z_sampling')


t = Dense(latent_dim, activation='tanh', name='decoder_hidden')(latent_inputs)
t = BatchNormalization()(t)
t = Dense(hidden_size5, activation='tanh' , name='decoder_hidden2')(t)
t = Dense(hidden_size4, activation='tanh' , name='decoder_hidden3')(t)
t = Dense(hidden_size3, activation='tanh' , name='decoder_hidden4')(t)
t = Dense(hidden_size2, activation='tanh', name='decoder_hidden5')(t)
t = Dense(hidden_size, activation='tanh', name='decoder_hidden6')(t)
decoded_mean = Dense(input_dim, activation='tanh', name='decoded_mean')(t)



decoder = Model(latent_inputs, decoded_mean)
decoder.summary()

outputs = decoder(encoder(x)[2])

#decoder = Model(latent_inputs, outputs)
#decoder.summary()

vae = Model(x, outputs)

def rec_loss(y_true, y_pred):
    return K.sum(K.square(y_true - y_pred), axis=-1)

def kl_loss(y_true, y_pred):
    return - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)

def vae_loss(x, decoded_mean):
    rec_loss = K.sum(K.square(x - decoded_mean), axis=-1)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return K.mean((rec_loss + kl_loss) / 2)

vae.compile(optimizer=Adam(lr=1e-2), loss=vae_loss, metrics=[rec_loss, kl_loss])
vae.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 352)          0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 352)          1408        input_1[0][0]                    
__________________________________________________________________________________________________
encoder_hidden5 (Dense)         (None, 123)          43419       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
encoder_hidden4 (Dense)         (None, 60)           7440        encoder_hidden5[0][0]            
__________________________________________________________________________________________________
encoder_hi

In [4]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

n_epochs = 100
batch_size = 128

early_stopping = EarlyStopping(monitor='loss', patience=10, min_delta=1e-5) #stop training if loss does not decrease with at least 0.00001
reduce_lr = ReduceLROnPlateau(monitor='loss', patience=5, min_delta=1e-5, factor=0.2) #reduce learning rate (divide it by 5 = multiply it by 0.2) if loss does not decrease with at least 0.00001

callbacks = [early_stopping, reduce_lr]

#collect training data in history object
history = vae.fit(x_train, x_train, 
                  validation_data=(x_val, x_val), 
                  batch_size=batch_size, epochs=n_epochs, 
                  callbacks=callbacks)



z_mean, z_log_var, z = encoder.predict(x_test,batch_size=128)

#for i, yi in enumerate(encode_set):
 #   print (yi)

#decoder_set = decoder.predict(encode_set, batch_size=128)

#print (decoder_set)

#encode_set.shape
print (z)



print (z.shape)
print (z_mean.shape)
print (z_log_var.shape)


decoded_sam = decoder.predict(z)
print (decoded_sam)
print (decoded_sam.shape)


decoded_sam2 = decoder.predict(z_mean)
print (decoded_sam2)
print (decoded_sam2.shape)

decoded_sam3 = decoder.predict(z_log_var)
print (decoded_sam3)
print (decoded_sam3.shape)

z_mean = shuffle(z_mean)
decoded_sam4 = decoder.predict(z_mean)
print (decoded_sam4)
print (decoded_sam4.shape)

z = shuffle(z)
decoded_sam5 = decoder.predict(z)
print (decoded_sam5)
print (decoded_sam5.shape)

arr = np.array(decoded_sam)
arr2 = np.array(decoded_sam2)
arr3 = np.array(decoded_sam3)
arr4 = np.array(decoded_sam4)
arr5 = np.array(decoded_sam5)

#arr = np.array(arr)
#samples=[]
columnsTitles = column

#arr = transpose(arr)
#arr.shape
print (arr)


columnsTls= columnsTitles[:-1]
#print (columnsTls)

df = pd.DataFrame(arr, columns=columnsTls )
print (df.head(10))
df2 = pd.DataFrame(arr2, columns=columnsTls )
print (df2.head(10))
df3 = pd.DataFrame(arr3, columns=columnsTls )
print (df3.head(10))
df4 = pd.DataFrame(arr4, columns=columnsTls )
print (df4.head(10))
df5 = pd.DataFrame(arr5, columns=columnsTls )
print (df5.head(10))
df = df.append(df2)
df = df.append(df3)
df = df.append(df4)
df = df.append(df5)

df = shuffle(df)
print (df.head(10))
print (df.shape)
df['TARGET'] = 1.0
print (df.head(10))
print (df.shape)

#norm = pd.read_csv("train(new)2_normalize_new2.csv")

#normt = norm.astype('float32').copy()
#train_data = shuffle(normt)
#print (train_data.head(10))

print(data_df3.TARGET.value_counts())
column = data_df3.columns.values.tolist()


df = df.reindex(columns=columnsTitles)
data_df3 =data_df3.append(df)
#train_data = shuffle(train_data)
print (data_df3.head(10))
print (data_df3.shape)

print(data_df.TARGET.value_counts())

data_df.to_csv(r"\data\new_dataset3_train3_normalized_balance_new.csv", encoding='utf-8', sep=',', index= False)